# Simulator Testing 0

The following code defines an ellipse corresponding to the path around the globe an orbit takes

In [96]:
import numpy as np

class Ellipse:
    def __init__(self, centre, height, width, angle=0):
        self.x, self.y = centre
        self.w = width
        self.h = height
        self.theta = angle
    
    def r(self):
        """
        Defines the rotation matrix used to transform coordinates to 0-centred model
        """
        a = np.cos(self.theta)
        b = np.sin(self.theta)
        return np.array([[a,-b],[b,a]])
    
    def point_at_angle(self, angle):
        """
        Given an interior angle from an ellipse's centre, finds a point on the ellipse.
        """
        #Compute location of original point
        x_or = 0.5 * self.w * np.cos(angle)
        y_or = 0.5 * self.w * np.sin(angle)

        #rotate the point to the centred version of the ellipse
        rot_point = np.dot(self.r(), [x_or, y_or]) + [self.x, self.y]
        return rot_point
    
    def multi_points(self, locations):
        """
        Given an array of interior angles, this returns an array of positions on the ellipse.
        """
        points_array = []
        for loc in locations:
            points_array.append(self.point_at_angle(self,loc))
        return np.array(points_array)
    
    def closest_point(self, x, tol=1e-6, max_iter = 100):
        """
        Given a point outside the ellipse, and finds the point on the ellipse closest to this.
        It also returns the (smallest) distance between the point and the ellipse.
        """
        #First need to project the point to the space where ellipse centred at origin
        #Also, need to initialize the constants
        x = np.asarray(x)
        x2 = np.dot(self.r().T, x - [self.x, self.y])
        d0 = np.arctan2(x2[1], x2[0])
        a = 0.5 * self.w
        b = 0.5 * self.h
        #Check if points are in the interior of the ellipse
        if (x2[0] / a)**2 + (x2[1] / b)**2 <1:
            print("Initial point in interior!")
            return np.array(0), None
        
        iterations = 0
        ds = [d0]
        xs = []
        errors = [tol]

        while errors[-1] >= tol and iterations < max_iter:
            x1 = np.array([a * np.cos(ds[-1]), b * np.sin(ds[-1])])
            xs.append(x1)
            dx1dt = np.array([-a *np.sin(ds[-1]), b * np.cos(ds[-1])])
            dx1dt2 = np.array([-a * np.cos(ds[-1]), -b * np.sin(ds[-1])])
            dD2dt = 2*np.dot(dx1dt,(x1 - x2))
            dD2dt2 = 2*np.dot(dx1dt2,(x1-x2)) + 2*np.dot(dx1dt,dx1dt)
            ds.append(ds[-1] - dD2dt / dD2dt2)
            errors.append(np.abs(dD2dt/dD2dt2))
            iterations +=1
        
        dist = np.linalg.norm(x1 - x2)
        return dist, xs[-1]




In [97]:
#Some quick tests
ellipse1 = Ellipse((0,0),10,20, np.deg2rad(0))
print(ellipse1.closest_point([14,14]))

(12.511067971689593, array([7.63442312, 3.22938011]))


To Be Fixed: currently only keeps track of position for (0,0) centred ellipse coords! Needs to be fixed!

In [119]:
#Here are some global variables
M_e = 5.972e24
G = 6.673e-11


In [143]:
class Satellite:
    def __init__(self, mass, dragcoeff, init_position, init_veloc):
        """
        Begin by defining relevant attributes of a satellite, initial poisition,
        and set up arrays to track movement of satellite.
        """
        self.mass = mass
        self.dragcoeff = dragcoeff
        self.position = init_position
        self.velocity = init_veloc
        self.poshist = [init_position]
        self.velochist = [init_veloc]

    def calcAltitude(self, ell: Ellipse):
        """
        Given a satellite and an ellipse, this performs the same calculation as the ellipse
        But more from a "Sattelite" POV
        """
        altitude, ground_pos = ell.closest_point(self.position)
        return altitude, ground_pos
    
    def calcGravAcc(self, ell: Ellipse):
        """
        Given a satellite's position, and accounting for non-zero-centred ellipses, computes
        The gravitational accelleration experienced by the satellite at that position
        """
        print(np.linalg.norm(np.array(self.position) - np.array([ell.x, ell.y]))**2)
        grav_acc = -G*M_e / np.linalg.norm(np.array(self.position) - np.array([ell.x, ell.y]))**2
        return np.array([0,grav_acc])

    def calcDrag(self, altitude):
        """
        Ideally takes a reference from the physical parameters team to estimate drag
        """
        rho_0 = 1.225
        H = 8400.0
        A = np.pi * (self.dragcoeff / 2)**2

        cur_veloc = self.velochist[-1]
        rho = rho_0 * np.exp(-(altitude) / H)
        Fd_x = 0.5 * rho * cur_veloc[0]**2 * self.dragcoeff * A
        Fd_y = 0.5 * rho * cur_veloc[1]**2 * self.dragcoeff * A
        return np.array([Fd_x, Fd_y])
    
    def FEtimeStep(self, ell: Ellipse, dt):
        """
        Takes a timestep using Forward Euler
        """
        self.altitude, groundpos = self.calcAltitude(ell)
        drag_a = self.calcDrag(self.altitude) / self.mass
        grav_a = self.calcGravAcc(ell)
        tot_a = drag_a + grav_a
        new_veloc = self.velochist[-1] + dt * tot_a
        self.velocity = new_veloc
        self.velochist.append(new_veloc)
        new_position = self.poshist[-1] + dt * tot_a
        self.position = new_position
        self.poshist.append(new_veloc)
    
    def forecast(self, ell: Ellipse, dt=0.01, maxIter = 10000):
        self.altitude, groundpos = self.calcAltitude(ell)
        iter = 0
        while self.altitude > 0 and iter < maxIter:
            self.FEtimeStep(ell, dt)
            #print(self.position)
        return self.poshist, self.velochist


In [144]:
sat1 = Satellite(3000, 2.2, [7378, 0], [3,0])
ell1 = Ellipse((0,0), 6378*2, 6378*2)

In [145]:
#print(ell1.closest_point([7378, 0]))
poshist, velochist = sat1.forecast(ell1)

54434884.0
5413971525.521266
5463315489.046645
5567190952.421012
5670277290.723289
5772595274.797815
5874164742.980884
5975004666.218721
6075133206.469358
6174567769.312866
6273325051.541384
6371421084.3756895
6468871272.853333
6565690431.849781
6661892819.125052
6757492165.731373
6852501704.069782
6946934193.843997
7040801946.1265745
7134116845.724394
7226890372.006721
7319133618.339007
7410857310.248443
7502071822.432507
7592787194.70922
7683013146.996875
7772759093.4015875
7862034155.482872
7950847174.760224
8039206724.517587
8127121120.956942
8214598433.747576
8301646496.0132065
8388272913.795463
8474485075.028803
8560290158.058963
8645695139.734394
8730706803.097689
8815331744.701935
8899576381.574839
8983446957.851892
9066949551.09816
9150090078.336792
9232874301.801153
9315307834.42625
9397396145.093927
9479144563.64551
9560558285.674438
9641642377.110819
9722401778.608847
9802841309.747469
9882965673.05398
9962779457.859646
10042287143.995758
10121493105.33829
10200401613.20856

KeyboardInterrupt: 

In [102]:
poshist

[[7378, 0]]